### Designing and Creating Database using SQLite and Python


In this project, we're going to :

- Import data into SQLite
- Design a normalized database schema
- Create tables for our schema
- Insert data into our schema

We will be working with a file of Major League Baseball games from Retrosheet. Retrosheet compiles detailed statistics on baseball games from the 1800s through to today. 

The main file we will be working from **game_log.csv**

In addition to the main file, we have also included three 'helper' files, also sourced from Retrosheet:

- park_codes.csv
- person_codes.csv
- team_codes.csv

These three helper files in some cases contain extra data.

**game_log_fields.txt** file from Retrosheet which explains the fields included in main file, will be useful to assist in exploratory data analysis

In [34]:
import pandas as pd
import re
import sqlite3 as s3
pd.set_option('max_columns', 180)
pd.set_option('max_rows', 200000)
pd.set_option('max_colwidth', 5000)

In [35]:
!cat game_log_fields.txt

Field(s)  Meaning
    1     Date in the form "yyyymmdd"
    2     Number of game:
             "0" -- a single game
             "1" -- the first game of a double (or triple) header
                    including seperate admission doubleheaders
             "2" -- the second game of a double (or triple) header
                    including seperate admission doubleheaders
             "3" -- the third game of a triple-header
             "A" -- the first game of a double-header involving 3 teams
             "B" -- the second game of a double-header involving 3 teams
    3     Day of week  ("Sun","Mon","Tue","Wed","Thu","Fri","Sat")
  4-5     Visiting team and league
    6     Visiting team game number
          For this and the home team game number, ties are counted as
          games and suspended games are counted from the starting
          rather than the ending date.
  7-8     Home team and league
    9     Home team game number
10-11     Visiting and home team score (unquoted)


In [36]:
csv_files = ["game_log.csv", "park_codes.csv", "person_codes.csv", "team_codes.csv"]

dfs = {re.sub(".csv","",file) : pd.read_csv(file, index_col=0) for file in csv_files}

/dataquest/system/env/python3/lib/python3.4/site-packages/IPython/core/interactiveshell.py:2825: DtypeWarning: Columns (12,13,14,15,19,20,81,82,83,84,85,86,87,88,93,94,95,96,97,98,99,100,105,106,108,109,111,112,114,115,117,118,120,121,123,124,126,127,129,130,132,133,135,136,138,139,141,142,144,145,147,148,150,151,153,154,156,157,160) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [37]:
for key,value in dfs.items():
    print(key)
    print(value.shape)
    print(value.head())

person_codes
(20494, 6)
             last   first player_debut mgr_debut coach_debut ump_debut
id                                                                    
aardd001  Aardsma   David   04/06/2004       NaN         NaN       NaN
aaroh101    Aaron    Hank   04/13/1954       NaN         NaN       NaN
aarot101    Aaron  Tommie   04/10/1962       NaN  04/06/1979       NaN
aased001     Aase     Don   07/26/1977       NaN         NaN       NaN
abada001     Abad    Andy   09/10/2001       NaN         NaN       NaN
team_codes
(150, 7)
        league  start   end       city         nickname franch_id  seq
team_id                                                               
ALT         UA   1884  1884    Altoona  Mountain Cities       ALT    1
ARI         NL   1998     0    Arizona     Diamondbacks       ARI    1
BFN         NL   1879  1885    Buffalo           Bisons       BFN    1
BFP         PL   1890  1890    Buffalo           Bisons       BFP    1
BL1        NaN   1872  1874  Balt

### Description of each file

- **game_log.csv** - This is the main log fine which has complete statistics of the game, home and visiting teams and its players, venue, umpires, offensive and defensive stats of the team in that game.

- **park_codes.csv** - This file contains information about the parks/stadium where the games are held. **park_id** can be used for joining with main file.

- **person_codes.csv** - This file contains info like ID, name, debut date about the players/coaches/umpires. **id** can be used for joining with main file.

- **team_codes.csv** - This file contains info like team ID, league the team belong to, city, frachise info etc. **team_id** can be used for joining with main file.

In [38]:
print(dfs['game_log']["v_league"].unique())
print(dfs['game_log']["h_league"].unique())
print(dfs['team_codes']["league"].unique())
print(dfs['park_codes']["league"].unique())

[nan 'NL' 'AA' 'UA' 'PL' 'AL' 'FL']
[nan 'NL' 'AA' 'UA' 'PL' 'AL' 'FL']
['UA' 'NL' 'PL' nan 'AA' 'AL' 'FL']
['NL' 'UA' 'AL' nan 'AA' 'FL' 'PL']


### Notes on defensive positions and various league fields

- **Defensive positions** - There are nine fielding positions in baseball. Each position conventionally has an associated number, which is used to score putouts: 1 (pitcher), 2 (catcher), 3 (first baseman), 4 (second baseman), 5 (third baseman), 6 (shortstop), 7 (left fielder), 8 (center fielder), and 9 (right fielder).
    
    
- **LEAGUES** - 
    NL: National League 
    AA: American Association 
    FL: Federal League 
    UA: Union Association 
    PL: Players' League
    AL: American League

In [39]:
def run_query(q):
    with s3.connect('mlb.db') as conn:
        return pd.read_sql(q, conn)
    
def run_command(c):
    with s3.connect('mlb.db') as conn:
        conn.execute('PRAGMA foreign_keys = ON;')
        conn.isolation_level = None
        conn.execute(c)
        
def show_tables():
    q = """SELECT
              name,
              type
           FROM sqlite_master
           WHERE type IN ("table","view");"""
    return run_query(q)

In [40]:
# Code to drop DB if ran into any issues (should exeucte it in seperate cell)

'''
!rm mlb.db
'''

In [41]:
for key, value in dfs.items():
    with s3.connect('mlb.db') as conn:
        conn.execute('DROP TABLE IF EXISTS {};'.format(key))
        value.to_sql(key, conn)

In [42]:
c1 =''' 
    ALTER TABLE game_log
    ADD COLUMN game_id VARCHAR(13);
    '''
try:
    run_command(c1)
except:
    pass


c2 = '''
    UPDATE game_log
    SET game_id = date || h_name || number_of_game
    WHERE game_id IS NULL;
    '''
# WHERE clause prevents this if it has already been done

run_command(c2)

q = '''
    SELECT
        game_id,
        h_name,
        date,
        number_of_game
    FROM game_log
        LIMIT 5;
    '''

run_query(q)

,game_id,h_name,date,number_of_game
0,18710504FW10,FW1,18710504,0
1,18710505WS30,WS3,18710505,0
2,18710506RC10,RC1,18710506,0
3,18710508CH10,CH1,18710508,0
4,18710509TRO0,TRO,18710509,0


### Normalizing and creating the schema
#### Tables
- person
- park
- league
- appearance_type

### Creating person table

In [43]:
c1 = '''
    CREATE TABLE IF NOT EXISTS person (
    person_id VARCHAR(10) PRIMARY KEY,
    first_name TEXT,
    last_name TEXT
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO person
        SELECT
            id,
            first,
            last
        FROM person_codes;
    '''

q = '''SELECT * FROM person LIMIT 5;'''

run_command(c1)
run_command(c2)
run_query(q)

,person_id,first_name,last_name
0,aardd001,David,Aardsma
1,aaroh101,Hank,Aaron
2,aarot101,Tommie,Aaron
3,aased001,Don,Aase
4,abada001,Andy,Abad


### Creating park table

In [44]:
c1 = '''
    CREATE TABLE IF NOT EXISTS park (
    park_id VARCHAR (10) PRIMARY KEY,
    name TEXT,
    nickname TEXT,
    city TEXT,
    state TEXT,
    notes TEXT
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO park
        SELECT
            park_id,
            name,
            aka,
            city,
            state,
            notes
        FROM park_codes;
    '''

q = '''SELECT * FROM park LIMIT 5;'''

run_command(c1)
run_command(c2)
run_query(q)

,park_id,name,nickname,city,state,notes
0,ALB01,Riverside Park,None,Albany,NY,TRN:9/11/80;6/15&9/10/1881;5/16-5/18&5/30/1882
1,ALT01,Columbia Park,None,Altoona,PA,None
2,ANA01,Angel Stadium of Anaheim,Edison Field; Anaheim Stadium,Anaheim,CA,None
3,ARL01,Arlington Stadium,None,Arlington,TX,None
4,ARL02,Rangers Ballpark in Arlington,The Ballpark in Arlington; Ameriquest Fl,Arlington,TX,None


### Creating league table

In [45]:
c1 = '''
    CREATE TABLE IF NOT EXISTS league (
    league_id TEXT PRIMARY KEY,
    name TEXT
    );
    '''
c2 = '''
    INSERT OR IGNORE INTO league
        VALUES
        ("NL","National League"),
        ("AA","American Association"),
        ("FL","Federal League"),
        ("UA","Union Association"),
        ("PL","Players' League"),
        ("AL","American League");
    '''

q = '''SELECT * FROM league;'''

run_command(c1)
run_command(c2)
run_query(q)

,league_id,name
0,NL,National League
1,AA,American Association
2,FL,Federal League
3,UA,Union Association
4,PL,Players' League
5,AL,American League


### Creaing appearance_type table

In [46]:
c1 = '''DROP TABLE IF EXISTS appearance_type'''

run_command(c1)

c2 = '''
    CREATE TABLE IF NOT EXISTS appearance_type (
    appearance_type_id TEXT PRIMARY KEY,
    name TEXT,
    category TEXT
    );
    '''

appearance_type_csv = pd.read_csv("appearance_type.csv")

with s3.connect('mlb.db') as conn:
    appearance_type_csv.to_sql("appearance_type_csv",
                           conn, index=False,
                           if_exists='append')
    
c3 = '''
    INSERT OR IGNORE INTO appearance_type
        SELECT
            appearance_type_id,
            name,
            category
        FROM appearance_type_csv
    '''
    
q = '''SELECT * FROM appearance_type'''

run_command(c2)
run_command(c3)
run_query(q)

,appearance_type_id,name,category
0,O1,Batter 1,offense
1,O2,Batter 2,offense
2,O3,Batter 3,offense
3,O4,Batter 4,offense
4,O5,Batter 5,offense
5,O6,Batter 6,offense
6,O7,Batter 7,offense
7,O8,Batter 8,offense
8,O9,Batter 9,offense
9,D1,Pitcher,defense


### Creating team table

In [47]:
c1 = '''
    CREATE TABLE IF NOT EXISTS team (
    team_id VARCHAR(5) PRIMARY KEY,
    league_id VARCHAR (5),
    city TEXT,
    nickname TEXT,
    franch_id VARCHAR (5),
        FOREIGN KEY (league_id) REFERENCES league(league_id)
    );
    '''

c2 = '''
    INSERT OR IGNORE INTO team
        SELECT
            team_id,
            league,
            city,
            nickname,
            franch_id
        FROM team_codes;
    '''


q = '''SELECT * FROM team LIMIT 5'''


run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


### Adding The Team and Game Tables

In [48]:
c1 = """
CREATE TABLE IF NOT EXISTS team (
    team_id TEXT PRIMARY KEY,
    league_id TEXT,
    city TEXT,
    nickname TEXT,
    franch_id TEXT,
    FOREIGN KEY (league_id) REFERENCES league(league_id)
);
"""

c2 = """
INSERT OR IGNORE INTO team
SELECT
    team_id,
    league,
    city,
    nickname,
    franch_id
FROM team_codes;
"""

q = """
SELECT * FROM team
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,team_id,league_id,city,nickname,franch_id
0,ALT,UA,Altoona,Mountain Cities,ALT
1,ARI,NL,Arizona,Diamondbacks,ARI
2,BFN,NL,Buffalo,Bisons,BFN
3,BFP,PL,Buffalo,Bisons,BFP
4,BL1,None,Baltimore,Canaries,BL1


In [49]:
c1 = """
CREATE TABLE IF NOT EXISTS game (
    game_id TEXT PRIMARY KEY,
    date TEXT,
    number_of_game INTEGER,
    park_id TEXT,
    length_outs INTEGER,
    day BOOLEAN,
    completion TEXT,
    forefeit TEXT,
    protest TEXT,
    attendance INTEGER,
    legnth_minutes INTEGER,
    additional_info TEXT,
    acquisition_info TEXT,
    FOREIGN KEY (park_id) REFERENCES park(park_id)
);
"""

c2 = """
INSERT OR IGNORE INTO game
SELECT
    game_id,
    date,
    number_of_game,
    park_id,
    length_outs,
    CASE
        WHEN day_night = "D" THEN 1
        WHEN day_night = "N" THEN 0
        ELSE NULL
        END
        AS day,
    completion,
    forefeit,
    protest,
    attendance,
    length_minutes,
    additional_info,
    acquisition_info
FROM game_log;
"""

q = """
SELECT * FROM game
LIMIT 5;
"""

run_command(c1)
run_command(c2)
run_query(q)

,game_id,date,number_of_game,park_id,length_outs,day,completion,forefeit,protest,attendance,legnth_minutes,additional_info,acquisition_info
0,18710504FW10,18710504,0,FOR01,54,1,None,None,None,200,120,None,Y
1,18710505WS30,18710505,0,WAS01,54,1,None,None,None,5000,145,HTBF,Y
2,18710506RC10,18710506,0,RCK01,54,1,None,None,None,1000,140,None,Y
3,18710508CH10,18710508,0,CHI01,54,1,None,None,None,5000,150,None,Y
4,18710509TRO0,18710509,0,TRO01,54,1,None,None,None,3250,145,HTBF,Y


### Adding the Team Appearance Table

In [50]:
c1 = """
CREATE TABLE IF NOT EXISTS team_appearance (
    team_id TEXT,
    game_id TEXT,
    home BOOLEAN,
    league_id TEXT,
    score INTEGER,
    line_score TEXT,
    at_bats INTEGER,
    hits INTEGER,
    doubles INTEGER,
    triples INTEGER,
    homeruns INTEGER,
    rbi INTEGER,
    sacrifice_hits INTEGER,
    sacrifice_flies INTEGER,
    hit_by_pitch INTEGER,
    walks INTEGER,
    intentional_walks INTEGER,
    strikeouts INTEGER,
    stolen_bases INTEGER,
    caught_stealing INTEGER,
    grounded_into_double INTEGER,
    first_catcher_interference INTEGER,
    left_on_base INTEGER,
    pitchers_used INTEGER,
    individual_earned_runs INTEGER,
    team_earned_runs INTEGER,
    wild_pitches INTEGER,
    balks INTEGER,
    putouts INTEGER,
    assists INTEGER,
    errors INTEGER,
    passed_balls INTEGER,
    double_plays INTEGER,
    triple_plays INTEGER,
    PRIMARY KEY (team_id, game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id)
);
"""

run_command(c1)

c2 = """
INSERT OR IGNORE INTO team_appearance
    SELECT
        h_name,
        game_id,
        1 AS home,
        h_league,
        h_score,
        h_line_score,
        h_at_bats,
        h_hits,
        h_doubles,
        h_triples,
        h_homeruns,
        h_rbi,
        h_sacrifice_hits,
        h_sacrifice_flies,
        h_hit_by_pitch,
        h_walks,
        h_intentional_walks,
        h_strikeouts,
        h_stolen_bases,
        h_caught_stealing,
        h_grounded_into_double,
        h_first_catcher_interference,
        h_left_on_base,
        h_pitchers_used,
        h_individual_earned_runs,
        h_team_earned_runs,
        h_wild_pitches,
        h_balks,
        h_putouts,
        h_assists,
        h_errors,
        h_passed_balls,
        h_double_plays,
        h_triple_plays
    FROM game_log

UNION

    SELECT    
        v_name,
        game_id,
        0 AS home,
        v_league,
        v_score,
        v_line_score,
        v_at_bats,
        v_hits,
        v_doubles,
        v_triples,
        v_homeruns,
        v_rbi,
        v_sacrifice_hits,
        v_sacrifice_flies,
        v_hit_by_pitch,
        v_walks,
        v_intentional_walks,
        v_strikeouts,
        v_stolen_bases,
        v_caught_stealing,
        v_grounded_into_double,
        v_first_catcher_interference,
        v_left_on_base,
        v_pitchers_used,
        v_individual_earned_runs,
        v_team_earned_runs,
        v_wild_pitches,
        v_balks,
        v_putouts,
        v_assists,
        v_errors,
        v_passed_balls,
        v_double_plays,
        v_triple_plays
    from game_log;
"""

run_command(c2)

q = """
SELECT * FROM team_appearance
WHERE game_id = (
                 SELECT MIN(game_id) from game
                )
   OR game_id = (
                 SELECT MAX(game_id) from game
                )
ORDER By game_id, home;
"""

run_query(q)

,team_id,game_id,home,league_id,score,line_score,at_bats,hits,doubles,triples,homeruns,rbi,sacrifice_hits,sacrifice_flies,hit_by_pitch,walks,intentional_walks,strikeouts,stolen_bases,caught_stealing,grounded_into_double,first_catcher_interference,left_on_base,pitchers_used,individual_earned_runs,team_earned_runs,wild_pitches,balks,putouts,assists,errors,passed_balls,double_plays,triple_plays
0,CL1,18710504FW10,0,None,0,000000000,30,4,1,0,0,0,0,0,0,1,NaN,6,1,NaN,-1,NaN,4,1,1,1,0,0,27,9,0,3,0,0
1,FW1,18710504FW10,1,None,2,010010000,31,4,1,0,0,2,0,0,0,1,NaN,0,0,NaN,-1,NaN,3,1,0,0,0,0,27,3,3,1,1,0
2,MIA,20161002WAS0,0,NL,7,000230020,38,14,1,1,2,7,1,0,0,3,2.0,10,1,1.0,1,0.0,8,7,10,10,1,0,24,11,0,0,1,0
3,WAS,20161002WAS0,1,NL,10,03023002x,30,10,2,0,1,10,1,1,1,8,0.0,3,2,0.0,1,0.0,7,6,7,7,1,0,27,11,0,0,1,0


### Creating person_appearance table

In [51]:
c1 = "DROP TABLE IF EXISTS person_appearance"

run_command(c1)

c2 = """
CREATE TABLE person_appearance (
    appearance_id INTEGER PRIMARY KEY,
    person_id TEXT,
    team_id TEXT,
    game_id TEXT,
    appearance_type_id,
    FOREIGN KEY (person_id) REFERENCES person(person_id),
    FOREIGN KEY (team_id) REFERENCES team(team_id),
    FOREIGN KEY (game_id) REFERENCES game(game_id),
    FOREIGN KEY (appearance_type_id) REFERENCES appearance_type(appearance_type_id)
);
"""

c3 = """
INSERT OR IGNORE INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        NULL,
        hp_umpire_id,
        "UHP"
    FROM game_log
    WHERE hp_umpire_id IS NOT NULL    

UNION

    SELECT
        game_id,
        NULL,
        [1b_umpire_id],
        "U1B"
    FROM game_log
    WHERE "1b_umpire_id" IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [2b_umpire_id],
        "U2B"
    FROM game_log
    WHERE [2b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        [3b_umpire_id],
        "U3B"
    FROM game_log
    WHERE [3b_umpire_id] IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        lf_umpire_id,
        "ULF"
    FROM game_log
    WHERE lf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        NULL,
        rf_umpire_id,
        "URF"
    FROM game_log
    WHERE rf_umpire_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_manager_id,
        "MM"
    FROM game_log
    WHERE v_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_manager_id,
        "MM"
    FROM game_log
    WHERE h_manager_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_pitcher_id,
        "AWP"
    FROM game_log
    WHERE winning_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score < v_score THEN h_name
            ELSE v_name
            END,
        losing_pitcher_id,
        "ALP"
    FROM game_log
    WHERE losing_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        saving_pitcher_id,
        "ASP"
    FROM game_log
    WHERE saving_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        CASE
            WHEN h_score > v_score THEN h_name
            ELSE v_name
            END,
        winning_rbi_batter_id,
        "AWB"
    FROM game_log
    WHERE winning_rbi_batter_id IS NOT NULL

UNION

    SELECT
        game_id,
        v_name,
        v_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE v_starting_pitcher_id IS NOT NULL

UNION

    SELECT
        game_id,
        h_name,
        h_starting_pitcher_id,
        "PSP"
    FROM game_log
    WHERE h_starting_pitcher_id IS NOT NULL;
"""

template = """
INSERT INTO person_appearance (
    game_id,
    team_id,
    person_id,
    appearance_type_id
) 
    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "O{num}"
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL

UNION

    SELECT
        game_id,
        {hv}_name,
        {hv}_player_{num}_id,
        "D" || CAST({hv}_player_{num}_def_pos AS INT)
    FROM game_log
    WHERE {hv}_player_{num}_id IS NOT NULL;
"""

run_command(c2)
run_command(c3)

for hv in ["h","v"]:
    for num in range(1,10):
        query_vars = {
            "hv": hv,
            "num": num
        }
        run_command(template.format(**query_vars))

In [52]:
print(run_query("SELECT COUNT(DISTINCT game_id) games_game FROM game"))
print(run_query("SELECT COUNT(DISTINCT game_id) games_person_appearance FROM person_appearance"))

q = """
SELECT
    pa.*,
    at.name,
    at.category
FROM person_appearance pa
INNER JOIN appearance_type at on at.appearance_type_id = pa.appearance_type_id
WHERE PA.game_id = (
                   SELECT max(game_id)
                    FROM person_appearance
                   )
ORDER BY team_id, appearance_type_id
"""

run_query(q)

   games_game
0      171907
   games_person_appearance
0                   171907


,appearance_id,person_id,team_id,game_id,appearance_type_id,name,category
0,1646109,porta901,None,20161002WAS0,U1B,First Base,umpire
1,1646108,onorb901,None,20161002WAS0,U2B,Second Base,umpire
2,1646107,kellj901,None,20161002WAS0,U3B,Third Base,umpire
3,1646110,tumpj901,None,20161002WAS0,UHP,Home Plate,umpire
4,1646111,brica001,MIA,20161002WAS0,ALP,Losing Pitcher,award
5,6716279,koeht001,MIA,20161002WAS0,D1,Pitcher,defense
6,4744553,telit001,MIA,20161002WAS0,D2,Catcher,defense
7,5589581,bourj002,MIA,20161002WAS0,D3,1st Base,defense
8,4462877,gordd002,MIA,20161002WAS0,D4,2nd Base,defense
9,5026229,pradm001,MIA,20161002WAS0,D5,3rd Base,defense


In [53]:
show_tables()

,name,type
0,person_codes,table
1,team_codes,table
2,game_log,table
3,park_codes,table
4,person,table
5,park,table
6,league,table
7,appearance_type_csv,table
8,appearance_type,table
9,team,table


### Drop old tables

In [55]:
old_tables = ["game_log", "park_codes", "team_codes", "person_codes", "appearance_type_csv"]

for table in old_tables:
    run_command('DROP TABLE IF EXISTS {}'.format(table))
    
show_tables()

,name,type
0,person,table
1,park,table
2,league,table
3,appearance_type,table
4,team,table
5,game,table
6,team_appearance,table
7,person_appearance,table
